# **문제**
TF-IDF 연구용 자료
지난 시간 X 컬럼값들을 모델의 입력으로 주고 학습해 y를 맞추는 모델 연습을 해봤습니다.
이번 데이터는 7,395개의 웹사이트 데이터이며 각각 27개의 feature (column) 정보를 가지고 있습니다.

* y 예측할 정답컬럼:  label, 각각의 사이트가 환경친화적 사이트면 1, 아니면 0 입니다. (타이타닉 데이터 Survived 와 비슷)
* X 모델입력컬럼: 이번 데이터는 boilerplate 컬럼 하나 만 사용하면 됩니다.  웹사이트의 content 텍스트 정보를 가지고 있습니다.

즉 boilerplate 컬럼만 TF-IDF 방법으로 분석해서 모델의 입력으로 넣어주는 방식을 연구해보시면 됩니다.

해보시다 막히면 질문 주세요!

 P.S. 이번 데이터는 tsv 데이터라서 pd.read_csv( ) 대신 pd.read_table( ) 함수를 사용하시면 됩니다.

# **Step 1. 데이터 불러오기 & import**

In [56]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline # 전처리 + 모델링 

from nltk.corpus import words

In [57]:
df = pd.read_table("../input/practice-0930/stumble_upon_evergreen.tsv")

# **Step 2. 데이터 형태 확인**

In [58]:
df.head()

In [59]:
df.dtypes

In [60]:
df.columns

# **Step3. 데이터 전처리**

In [61]:
X_column = ['boilerplate', 'alchemy_category',
       'alchemy_category_score', 'avglinksize', 'commonlinkratio_1',
       'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4',
       'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio',
       'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news',
       'lengthyLinkDomain', 'linkwordscore', 'news_front_page',
       'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url',
       'parametrizedLinkRatio', 'spelling_errors_ratio']
y_column = 'label'

In [62]:
input_data = df.drop(['url', 'urlid'], axis=1)
input_data

In [63]:
df2 = df.copy()

# <데이터 벡터화>

# 방법 1 : CountVectorizer

CountVectorizer 쓰기 위해 빈칸으로 split 단위 변경

boilerpalce 컬

aaa bbb  cc  ddd  

----

aaa   10000

bbb    9999

ccc    8800
...

----

zzzzz       201

zzzzz2       202

In [64]:
from sklearn.feature_extraction.text import CountVectorizer

# word count 순 top 200개만 생성
# 불용어 처리 a, an, the ..
cv = CountVectorizer(
    max_features = 10, stop_words='english')

features = cv.fit_transform(df2['boilerplate'])

# sparse matrix 여서 numpy.ndarray 원 형태로 보기 위해 todense() 사용
featrues = pd.DataFrame(features.todense())
featrues

# 방법 2 : tf-idf

CountVecotrizer보다 좋은 성능

* 단어 개수 분석 : CountVectorizer, TF-IDF 가능
* 데이터를 벡터화: TF-IDF만 가능 (CountVectorizer는 못함 )
* 나중에 더 벡터화를 정교하게 하는 방법 (TF-IDF보다 더 좋은 방법): Word Embedding

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    min_df = 3,             # 최소 3번은 나와야 포함
    ngram_range = (1,2),    # 단어 묶음 1~2개까지 인덱싱
    sublinear_tf = True)    # 칼럼마다 평균 분산으로 스케일링 처리

text = tfidf.fit_transform(input_data['boilerplate'])
text # 차원 축소

In [66]:
# dimensionality reduction
from sklearn.decomposition import TruncatedSVD

# -> n_dims = 3,4,5 (높은 성능) 2~10 사이로 실험 후 제일 좋은 dim을 찾기
n_dims = 3 
svd = TruncatedSVD(n_components = n_dims)
text_svd = svd.fit_transform(text)
text_svd # 7395 * 3

<TF-IDF 분석을 통해서,  문자열을 --> 벡터로 해석해줌>
- 0번데이터  ---> <0.192879, 0.010573, 0.107206> 벡터로 vectorization
- 1 번데이터  ---> <0.187195, 0.006924, 0.083037> 벡터로 vectorization
- ...
- 7394 번데이터  ---> <0.070938, 0.066807,  0.236807> 벡터로 vectorization


<벡터는 Similarity 분석>
- A데이터 < 1, 1, 0>
- B데이터 < 999 ,999, 999>
- C데이터  < 1, 0,0>


- A 랑 C 가 유사한거를 계산가능 vector distance

In [67]:
text_svd = pd.DataFrame(text_svd)
text_svd = text_svd.add_prefix('boilerplate')
text_svd

In [68]:
input_data = pd.concat([input_data, text_svd], axis=1)
input_data

In [69]:
# 데이터에 ? 있음 -> 처리 필요 (alchemy_category, alchemy_category_score 컬럼 등)
input_data = input_data.drop(['boilerplate'], axis=1)
input_data 

In [83]:
# X 컬럼 재정의
X_column = ['boilerplate0', 'boilerplate1', 'boilerplate2', 'alchemy_category',
       'alchemy_category_score', 'avglinksize', 'commonlinkratio_1',
       'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4',
       'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio',
       'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news',
       'lengthyLinkDomain', 'linkwordscore', 'news_front_page',
       'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url',
       'parametrizedLinkRatio', 'spelling_errors_ratio']
y_column = 'label'

In [84]:
# filling missing values
input_data = input_data.replace('?', -1) # 결측치 ? -> -1 로 변환, fillna 역할

In [86]:
# label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in input_data.columns[input_data.dtypes == object]:
    input_data[i] = le.fit_transform(list(input_data[i]))

In [87]:
input_data.dtypes # 모두 숫자 형태로 바뀐 dataset

# **Learning & Classfication**

In [ ]:
# 모델 학습
# 1. train data  학습
#      - train data
#      - validation data 학습 하는 도중에 실행하는 성능평가
# 2. test data   모델 성능 평가

#전체 데이터 20개
# 16개 -> train (12), validation (4) 
# 4개 -> test

In [104]:
# 데이터 -> train/vaild, test
from sklearn.model_selection import train_test_split

tr_val_X, test_X, tr_val_y, test_y = train_test_split(
    input_data[X_column],   # X 입력데이터
    input_data[y_column],   # y 정답데이터
    test_size=0.2,
    shuffle=True,
    random_state=42,
    stratify=input_data[y_column]   # classification 일때는 중요한 옵션
    )
test_X

In [100]:
# 학습 도중에 하는 성능 평가
tr_val_data = pd.concat([tr_val_X, tr_val_y], axis=1)
tr_val_data

In [101]:
# 데이터 -> train/valid  ,  test
from sklearn.model_selection import train_test_split

train_X, valid_X, train_y, valid_y = train_test_split(
    tr_val_data[X_column],   # X 입력데이터
    tr_val_data[y_column],   # y 정답데이터
    test_size=0.2,
    shuffle=True,
    random_state=42,
    stratify=tr_val_data[y_column]   # classification 일때는 중요한 옵션
    )

In [93]:
print(len(train_X))
print(len(valid_X))
print(len(test_X))
len(train_X) + len(valid_X) + len(test_X) # 20

In [94]:
# train model
from xgboost import XGBClassifier

xgb = XGBClassifier(learning_rate=0.01,
    max_depth=8,
    colsample_bytree=0.8,
    n_estimators=1000)
xgb.fit(train_X, train_y, eval_set=[(valid_X, valid_y)])

In [95]:
# classification prediction (예측)
xgb_pred = xgb.predict(test_X)
#pred_y = model_rf.predict(test_X)
xgb_pred

In [98]:
true_y = test_y
true_y

In [99]:
from sklearn.metrics import classification_report

print(classification_report(true_y, xgb_pred))